In [1]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig():
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from Text_summary.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Text_summary.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(path_to_yaml=config_filepath)
        self.params = read_yaml(path_to_yaml=params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.Model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            #config
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            ##params
            num_epochs=self.params.num_epochs,
            warmup_steps=self.params.warmup_steps,
            per_device_train_batch_size=self.params.per_device_train_batch_size,
            weight_decay=self.params.weight_decay,
            logging_steps=self.params.logging_steps,
            evaluation_strategy=self.params.evaluation_strategy,
            eval_steps=self.params.eval_steps,
            save_steps=self.params.save_steps,
            gradient_accumulation_steps=self.params.gradient_accumulation_steps
        )

        return model_trainer_config        

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

e:\Anaconda\envs\text-sum\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-15 00:30:11,941]: INFO: config: PyTorch version 2.3.0 available.]


In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        sequence_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
  
        dataset = load_from_disk(self.config.data_path)

        # training_args = TrainingArguments(
        #     output_dir=self.config.root_dir,
        #     num_train_epochs=self.config.num_epochs,
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay,
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy,
        #     eval_steps=self.config.eval_steps,
        #     save_steps=float(self.config.save_steps),
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 


        trainer = Trainer(
            model=model,
            args=trainer_args,
            data_collator=sequence_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            tokenizer=tokenizer
        )

        trainer.train()

        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus_dailymail_cnn"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [14]:
try:
    config_manager = ConfigurationManager()
    model_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-05-15 00:32:15,645]: INFO: common: yaml file: config\config.yaml loaded]
[2024-05-15 00:32:15,650]: INFO: common: yaml file: params.yaml loaded]
[2024-05-15 00:32:15,651]: INFO: common: Created directory: artifacts]
[2024-05-15 00:32:15,653]: INFO: common: Created directory: artifacts/Model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no